In [3]:
!pip install unsloth


[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


In [4]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 128000 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2025.2.5: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    GPU: NVIDIA RTX 2000 Ada Generation. Max memory: 15.6 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.2.5 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


In [6]:
from datasets import load_dataset

# 데이터셋 불러오기
dataset = load_dataset("aripos1/gorani_dataset")

# train, test 스플릿 확인 (train, test 두 가지로 나뉘어 있음)
print(dataset)  # {'train': ..., 'test': ...}

README.md:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

train.parquet:   0%|          | 0.00/7.19M [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/1.92M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/21662 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5416 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'target_language', 'metadata', 'text'],
        num_rows: 21662
    })
    test: Dataset({
        features: ['instruction', 'input', 'output', 'target_language', 'metadata', 'text'],
        num_rows: 5416
    })
})


In [7]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset["train"],  # ✅ 'train' split을 명확히 지정
    eval_dataset = dataset["test"].shuffle(seed=1234).select(range(50)),   # ✅ 'test' split을 명확히 지정
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 8,
        gradient_accumulation_steps = 1,
        warmup_steps = 5,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        num_train_epochs = 3,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
        evaluation_strategy = "steps",
        eval_steps = 50,
        # ✅ Checkpoint 설정 (중간 저장)
        save_steps = 100,  # 매 10 스텝마다 저장
        save_total_limit = 3,  # 최근 3개 체크포인트만 유지
    ),
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Map (num_proc=2):   0%|          | 0/21662 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/50 [00:00<?, ? examples/s]

In [8]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 21,662 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 8 | Gradient Accumulation steps = 1
\        /    Total batch size = 8 | Total steps = 8,124
 "-____-"     Number of trainable parameters = 22,544,384


Step,Training Loss,Validation Loss
50,0.847000,0.749322
100,0.573700,0.659370
150,0.516600,0.616240
200,0.877400,0.589030
250,0.553000,0.569103
300,0.527200,0.546306
350,0.637300,0.537983
400,0.779500,0.531919
450,0.660200,0.518632
500,0.588400,0.503358


Unsloth: Not an error, but LlamaForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient
/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:716: UserWarning: Unable to fetch remote file due to the following error (ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: fae467d9-1299-49c6-a9fc-9b8fad3392be)') - silently ignoring the lookup for the file config.json in unsloth/llama-3.2-1b-instruct-unsloth-bnb-4bit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:246: UserWarning: Could not find a config file in unsloth/llama-3.2-1b-instruct-unsloth-bnb-4bit - will assume that the vocabulary was not modified.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:716: UserWarning: Unable to fetch remote file due to the 

In [9]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from huggingface_hub import HfApi

model_name = "gorani-1b-cos4"  # 저장할 모델 이름
repo_id = f"haeun0420/{model_name}"  # 저장소 이름

# 모델과 토크나이저 저장
model.save_pretrained(repo_id)
tokenizer.save_pretrained(repo_id)

('haeun0420/gorani-1b-cos4/tokenizer_config.json',
 'haeun0420/gorani-1b-cos4/special_tokens_map.json',
 'haeun0420/gorani-1b-cos4/tokenizer.json')

In [15]:
!pip install python-dotenv


[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


In [19]:
from dotenv import load_dotenv
load_dotenv()
from huggingface_hub import login

HUGGINGFACE_TOKEN= os.getenv("HUGGING_FACE_TOKEN")
login(token="HUGGING_FACE_TOKEN")

In [20]:
# Hugging Face에 업로드
api = HfApi()
api.create_repo(repo_id, exist_ok=True)  # 저장소 생성
api.upload_folder(repo_id=repo_id, folder_path=repo_id)

  0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/90.2M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/haeun0420/gorani-1b-cos4/commit/aaf5fb53d4c639f6c1abbee8e74999b39c9e6d19', commit_message='Upload folder using huggingface_hub', commit_description='', oid='aaf5fb53d4c639f6c1abbee8e74999b39c9e6d19', pr_url=None, repo_url=RepoUrl('https://huggingface.co/haeun0420/gorani-1b-cos4', endpoint='https://huggingface.co', repo_type='model', repo_id='haeun0420/gorani-1b-cos4'), pr_revision=None, pr_num=None)